In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import files
files.upload()

Saving Astar.py to Astar.py
Saving cnn.py to cnn.py
Saving cube_model_naive.py to cube_model_naive.py
Saving train.py to train.py


{'Astar.py': b'"""Implementation of A* heuristic search algorithm"""\nimport numpy as np\nimport heapq\nimport jax\nfrom collections import deque\nfrom functools import partial\nfrom time import time\n\nfrom cube_model_naive import Cube\n# from cnn import cnn_init, cnn_apply\n\n\ndef astar(state, params, approximator):\n    # (value, steps, stateID, actions)\n    v = approximator(params, np.array([state]))\n    uid = 0\n    node_states = {uid: state}\n    node_values = {uid: float(v)}\n    H = [(-float(v), 0, uid, tuple())]\n    heapq.heapify(H)\n\n    _cube = Cube()\n    i = 0\n    while i < 1000:\n        _, steps, current_id, actions = heapq.heappop(H)\n        current = node_states[current_id]\n        children, _ = Cube.expand_state(current)\n        children_values = approximator(params, children)\n        for cval, cstate, cact in zip(children_values, children, range(12)):\n            uid += 1\n            node_states[uid] = cstate\n            node_values[uid] = (-float(cval) 

In [3]:
import time
import jax
from cube_model_naive import Cube as env
from train import train

In [4]:
seed = 0
rng = jax.random.PRNGKey(seed=seed)

# GLOBAL VARIABLES IN THE SCRIPT
########## >>>>>   step_size = 1e-3
########## >>>>>   decay_rate = 0.0
########## >>>>>   reg = 1e-4
########## >>>>>   clip_max_grad = 10.0

In [5]:
tic = time.time()

params, loss_history = train(rng, env,
                             batch_size=128,
                             num_epochs=30,
                             num_iterations=41,
                             num_samples=1000,
                             print_every=20,
                             episodes=1000,
                             k_max=15,
                             verbose=True,
                             params_save_path=None)

toc = time.time()
print("\n TRAINING TOOK {:.2f} minutes".format((toc - tic) / 60.0))

	(Iteration(1/41) took 25.848 seconds) iter_mean_loss = 0.694
	(Iteration(21/41) took 12.137 seconds) iter_mean_loss = 1.386
	(Iteration(41/41) took 12.396 seconds) iter_mean_loss = 0.805
(Epoch (1/30) took 526.295 seconds), epoch_mean_loss = 1.362
	(Iteration(1/41) took 12.268 seconds) iter_mean_loss = 2.528
	(Iteration(21/41) took 12.371 seconds) iter_mean_loss = 0.645
	(Iteration(41/41) took 12.151 seconds) iter_mean_loss = 0.531
(Epoch (2/30) took 521.121 seconds), epoch_mean_loss = 0.836
	(Iteration(1/41) took 12.371 seconds) iter_mean_loss = 2.085
	(Iteration(21/41) took 13.161 seconds) iter_mean_loss = 0.710
	(Iteration(41/41) took 12.438 seconds) iter_mean_loss = 0.602
(Epoch (3/30) took 520.786 seconds), epoch_mean_loss = 0.822
	(Iteration(1/41) took 12.268 seconds) iter_mean_loss = 2.143
	(Iteration(21/41) took 11.794 seconds) iter_mean_loss = 0.657
	(Iteration(41/41) took 11.909 seconds) iter_mean_loss = 0.542
(Epoch (4/30) took 509.945 seconds), epoch_mean_loss = 0.795
	(It

In [5]:
root = "/content/drive/My Drive/Colab Notebooks/"
params_save_path = root + "params_cnn.npy"

In [ ]:
jax.numpy.save(params_save_path, params)

In [ ]:
params = jax.numpy.load(params_save_path, allow_pickle=True)

In [6]:
import numpy as np

import Astar

# from fcnn import fc_net as model_fn
from cnn import conv_net as model_fn

init_fun, apply_fun = model_fn()

cube = env()

In [9]:
times_solved = 0
for it in range(100):
  cube.set_random_state(seed=it, scrambles=10)
  for s in range(100):
    children, _ = env.expand_state(cube._state)
    vals = apply_fun(params, children)
    act = int(np.argmax(vals))
    cube.step(act)

    if cube.is_solved():
      times_solved += 1
      print("solved the cube at step {} in iteration {}".format(s + 1, it + 1))
      break
print("Managed to solve 10-scrambled cube {} times".format(times_solved))

times_solved = 0
for it in range(100):
  cube.set_random_state(seed=it, scrambles=5)
  for s in range(100):
    children, _ = env.expand_state(cube._state)
    vals = apply_fun(params, children)
    act = int(np.argmax(vals))
    cube.step(act)

    if cube.is_solved():
      times_solved += 1
      print("solved the cube at step {} in iteration {}".format(s + 1, it + 1))
      break
print("Managed to solve 5-scrambled cube {} times".format(times_solved))

solved the cube at step 6 in iteration 15
solved the cube at step 6 in iteration 95
Managed to solve 10-scrambled cube 2 times
solved the cube at step 1 in iteration 15
solved the cube at step 5 in iteration 46
solved the cube at step 3 in iteration 91
solved the cube at step 5 in iteration 95
Managed to solve 5-scrambled cube 4 times


In [11]:
num_cubes = 100
scrambles = 5
start = time.time()
solved = 0
for it in range(num_cubes):
  cube.set_random_state(seed=it, scrambles=scrambles)
  res = Astar.astar(cube._state, params, apply_fun)
  if res != ():
    solved += 1
end = time.time()
print("Solved {}/{} cubes scrambled {} times in {:.0f} seconds.".format(solved, num_cubes, scrambles, (end-start)))

Solved 100/100 cubes scrambled 5 times in 18 seconds.


In [10]:
num_cubes = 100
scrambles = 10
start = time.time()
solved = 0
for it in range(num_cubes):
  cube.set_random_state(seed=it, scrambles=scrambles)
  res = Astar.astar(cube._state, params, apply_fun)
  if res != ():
    solved += 1
end = time.time()
print("Solved {}/{} cubes scrambled {} times in {:.0f} seconds.".format(solved, num_cubes, scrambles, (end-start)))


num_cubes = 100
scrambles = 15
start = time.time()
solved = 0
for it in range(num_cubes):
  cube.set_random_state(seed=it, scrambles=scrambles)
  res = Astar.astar(cube._state, params, apply_fun)
  if res != ():
    solved += 1
end = time.time()
print("Solved {}/{} cubes scrambled {} times in {:.0f} seconds.".format(solved, num_cubes, scrambles, (end-start)))

Solved 69/100 cubes scrambled 10 times in 434 seconds.
Solved 16/100 cubes scrambled 15 times in 873 seconds.


In [ ]:
tic = time.time()

params, loss_history = train(rng, env,
                             batch_size=128,
                             num_epochs=30,
                             num_iterations=41,
                             num_samples=1000,
                             print_every=20,
                             episodes=1000,
                             k_max=15,
                             verbose=True,
                             params_save_path=params_save_path)

toc = time.time()
print("\n TRAINING TOOK {:.2f} minutes".format((toc - tic) / 60.0))

	(Iteration(1/41) took 26.918 seconds) iter_mean_loss = 1.269
	(Iteration(21/41) took 12.440 seconds) iter_mean_loss = 0.471
	(Iteration(41/41) took 12.100 seconds) iter_mean_loss = 0.441
(Epoch (1/30) took 538.424 seconds), epoch_mean_loss = 0.568


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


	(Iteration(1/41) took 12.466 seconds) iter_mean_loss = 1.347
	(Iteration(21/41) took 11.919 seconds) iter_mean_loss = 0.497
	(Iteration(41/41) took 11.836 seconds) iter_mean_loss = 0.449
(Epoch (2/30) took 504.928 seconds), epoch_mean_loss = 0.595
	(Iteration(1/41) took 11.838 seconds) iter_mean_loss = 1.351
	(Iteration(21/41) took 11.731 seconds) iter_mean_loss = 0.491
	(Iteration(41/41) took 11.737 seconds) iter_mean_loss = 0.427
(Epoch (3/30) took 493.565 seconds), epoch_mean_loss = 0.590
	(Iteration(1/41) took 11.816 seconds) iter_mean_loss = 1.389
	(Iteration(21/41) took 11.944 seconds) iter_mean_loss = 0.489
	(Iteration(41/41) took 11.766 seconds) iter_mean_loss = 0.425
(Epoch (4/30) took 486.718 seconds), epoch_mean_loss = 0.590
	(Iteration(1/41) took 11.526 seconds) iter_mean_loss = 1.341
	(Iteration(21/41) took 11.739 seconds) iter_mean_loss = 0.487
	(Iteration(41/41) took 11.522 seconds) iter_mean_loss = 0.432
(Epoch (5/30) took 484.417 seconds), epoch_mean_loss = 0.563
	(It